In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import xception

from data_loader import StanfordDogLoader

In [2]:
batch_size = 32
img_height = 224
img_width = 224

loader = StanfordDogLoader(img_dim=[img_height, img_width])

train_ds, test_ds = loader.load()

[INFO] Fetched 20580 images
[INFO] Found 120 classes
[INFO] The first ten classes are
['n02085620-Chihuahua' 'n02085782-Japanese_spaniel'
 'n02085936-Maltese_dog' 'n02086079-Pekinese' 'n02086240-Shih-Tzu'
 'n02086646-Blenheim_spaniel' 'n02086910-papillon' 'n02087046-toy_terrier'
 'n02087394-Rhodesian_ridgeback' 'n02088094-Afghan_hound']
[INFO] Image count per class
[152, 185, 252, 149, 214, 188, 196, 172, 172, 239, 175, 195, 187, 171, 159, 153, 157, 148, 151, 218, 182, 187, 188, 196, 151, 200, 232, 160, 155, 164, 182, 172, 179, 169, 172, 185, 164, 157, 197, 202, 202, 197, 196, 180, 182, 154, 157, 155, 158, 206, 183, 156, 169, 186, 152, 151, 150, 171, 167, 152, 154, 161, 155, 153, 152, 150, 159, 150, 159, 151, 150, 150, 154, 150, 150, 152, 153, 154, 169, 157, 153, 150, 150, 152, 152, 150, 184, 168, 218, 151, 202, 151, 156, 152, 159, 156, 170, 150, 178, 192, 150, 209, 200, 210, 195, 213, 218, 219, 196, 158, 153, 181, 155, 151, 155, 159, 155, 156, 150, 169]
[INFO] The dataset split is
Tra

In [6]:
for img, label in train_ds.take(10):
    print(loader.short_class_names[label.numpy()])

toy_terrier
bloodhound
vizsla
clumber
German_shepherd
collie
Great_Pyrenees
Norwich_terrier
Chihuahua
groenendael


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(ds: tf.data.Dataset, augment=False):
    ds = ds.batch(batch_size)
    ds = ds.map(lambda X, y: (xception.preprocess_input(X), y), num_parallel_calls=AUTOTUNE)
        
    return ds.prefetch(buffer_size=AUTOTUNE)

test_ds = preprocess(test_ds)

In [6]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def image_example(image_string, label):
    feature = {
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

record_file = 'images.tfrecord'
with tf.io.TFRecordWriter(record_file) as writer:
  for image_batch, label_batch in test_ds:
    image_string = tf.io.serialize_tensor(image_batch)
    tf_example = image_example(image_string, label_batch)
    writer.write(tf_example.SerializeToString())

TypeError: <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([ 59,  55,  71, 112,  97, 104,  21,  92,  96,  22,  21, 101,  30,
        95,  39,  61,  51,  10,  12,  39,  93,  12,  86,  46,  38,  29,
        95,  14,  11,  49,  96,  25], dtype=int64)> has type <class 'tensorflow.python.framework.ops.EagerTensor'>, but expected one of: (<class 'int'>,)

In [5]:
model_path = os.path.join("models", "dog_v4.h5")
model = keras.models.load_model(model_path)

In [6]:
batch_new = next(iter(test_ds))
X_new = batch_new[0]
y_new = batch_new[1]
print(X_new.shape)
print(y_new.shape)
print()
print("Predictions using original model and 'tf.nn.softmax'")
print("-----------------------------------")
pred = model.predict(X_new)
pred = tf.nn.softmax(pred, axis=1).numpy()
prob = pred / 120
arg_max = np.argmax(pred, axis=1)
print(arg_max)
print(np.argpartition(prob, axis=1))
print("Grounding Truth (labels)")
print("-----------------------------------")
print(y_new.numpy())

(32, 224, 224, 3)
(32,)

Predictions using original model and 'tf.nn.softmax'
-----------------------------------
[ 90 100  39 113  98  35  27  46 115  53  25  37 102   1 106  95   4 116
  63 107  86   0 111  49  99  20 113  88  31  18  55  60]


TypeError: _argpartition_dispatcher() missing 1 required positional argument: 'kth'